In [13]:
import requests

url_api = "http://127.0.0.1:5000/"
end_point = "nlp_model/resources/classify"

params = {'workspace_id': 'dc1e7b3d-9137-4a20-a99c-d0d2029ef170-cedae',
          'sentence': ""}

def classify(sentence):
    params["sentence"] = sentence
    r = requests.get(url_api+end_point, params=params)
    return r.json()

In [14]:
classify("quero ir numa agência da CEDAE")

{'entities': [],
 'intents': [{'confidence': 0.6871778811988557, 'intent': 'Realizar_Denuncia'},
  {'confidence': 0.6732106239226163, 'intent': 'Localizar_AgenciasCEDAE'},
  {'confidence': 0.629248455847625, 'intent': 'Reportar_Vazamento'},
  {'confidence': 0.6153294003585437, 'intent': 'Encontrar_ValorTarifa'},
  {'confidence': 0.5891592604406546, 'intent': 'Reportar_FaltaDeAgua'},
  {'confidence': 0.5511919645768384, 'intent': 'Emitir_NadaConsta'},
  {'confidence': 0.5294107681639796, 'intent': 'Solicitar_Contato'},
  {'confidence': 0.46576385031258694, 'intent': 'Parcelar_Conta'},
  {'confidence': 0.3912627948951297, 'intent': 'Revisar_Conta'},
  {'confidence': 0.3423868686757756, 'intent': 'Consultar_MatriculaCPF'}]}

# Upload Data

In [3]:
import json

file = open('/home/cloves/Downloads/skill-CEDAE---skill-for-comparation-(do-not-delete).json', "r") 
workspace = json.load(file)
workspace["intents"][-1]

{'intent': 'Solicitar_Contato',
 'examples': [{'text': 'Posso ligar para a Cedae?'},
  {'text': 'Preciso do email de contato com a ouvidoria'},
  {'text': 'Quero entrar em contato com a Cedae'},
  {'text': 'Quero falar com um consultor'},
  {'text': 'Solicitar atendimento'},
  {'text': 'Tem algum telefone ?'},
  {'text': 'Transferir para um atendente humano'}]}

In [5]:
end_point = "nlp_model/intents/"

for intent in workspace["intents"]:
    r = requests.post(url_api+end_point, params=params, json=intent)
    r = r.json()
    if r["result"] == "created":
        print("Intent %s was created with id %s"%(intent["intent"],r["_id"]))
    else:
        print("Intent %s was not created"%intent["intent"])

Intent Bot_About was created with id 0965YncB_pG9ugNAbEJ0
Intent Bot_Capabilities was created with id 1d65YncB_pG9ugNAcUL3
Intent Consultar_ContasPagas was created with id 3t65YncB_pG9ugNAd0IF
Intent Consultar_MatriculaCPF was created with id 3965YncB_pG9ugNAfEIl
Intent Emitir_NadaConsta was created with id 4N65YncB_pG9ugNAgUJ3
Intent Emitir_SegundaViaConta was created with id 4t65YncB_pG9ugNAhkKP
Intent Encontrar_ValorTarifa was created with id 5N65YncB_pG9ugNAi0K1
Intent General_Ending was created with id xd65YncB_pG9ugNAkEPd
Intent General_Greetings was created with id x965YncB_pG9ugNAlUP3
Intent General_Negative_Feedback was created with id 0N65YncB_pG9ugNAm0MR
Intent General_Positive_Feedback was created with id 0d65YncB_pG9ugNAoEM3
Intent Localizar_AgenciasCEDAE was created with id 0t65YncB_pG9ugNApUNV
Intent OutOfScope_Jokes was created with id 0965YncB_pG9ugNAqkNw
Intent Parcelar_Conta was created with id 1d65YncB_pG9ugNAr0OU
Intent Realizar_Denuncia was created with id 2N65Ync

# Function to talk with Watson Model

In [6]:
import json
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

API_KEY = "nA36HqlqQkQSDf0EywIeYGPl0qI_nNvLqvBUI_zhhUmR" # os.environ['API_KEY']
SERVICE_URL = "https://api.us-south.assistant.watson.cloud.ibm.com/instances/12c017ac-1bb4-4364-8493-9cc9ba98aa26" # os.environ['SERVICE_URL']
ASSISTANT_ID = "eb61f20c-beb4-491f-a9e4-b36c65bcae9c" # os.environ['ASSISTANT_ID']

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)

def watson_predict(sentence):
    response = assistant.message(
        workspace_id=ASSISTANT_ID,
        input={
            'message_type': 'text',
            'text': sentence
        }
    ).get_result()
    if response.get("intents") and response["intents"]:
        return (response["intents"][0]["intent"], response["intents"][0]["confidence"])
    else:
        return "failed to identify", -1.0

# Battle Omni NLP API BM25 vs. Watson Assistent

In [7]:
import pandas as pd

df_test = pd.read_csv("data/test_data_cedae.csv", sep=",")
df_test.head()

,example,intent
0,quero ir numa agência da CEDAE,Localizar_AgenciasCEDAE
1,Boa Noite,General_Greetings
2,quero um relatório dos meus pagamentos,Consultar_ContasPagas
3,você sabe falar sobre o que,Bot_Capabilities
4,cobranca indevida,Revisar_Conta


In [15]:
omni_right_answer = 0
watson_right_answer = 0
for _, row in df_test.iterrows():
    omni_result = classify(row.example)
    watson_result = watson_predict(row.example)
    
    print("-------------------------------------------")
    print("sentence: %s"%row.example)
    print("\n")
    print("Omni NLP API\n")
    print("predict_intent: %s"%omni_result["intents"][0]["intent"])
    print("score: %.2f"%omni_result["intents"][0]["confidence"])
    print("right_intent: %s"%row.intent)
    if omni_result["intents"][0]["intent"] == row.intent:
        omni_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %omni_right_answer)
    print("\n")
    
    print("Watson Assistent\n")
    print("predict_intent: %s"%watson_result[0])
    print("score: %.2f"%watson_result[1])
    print("right_intent: %s"%row.intent)
    if watson_result[0] == row.intent:
        watson_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %watson_right_answer)
    print("\n")

print("-------------------------------------------\n")
print("Total of questions: %d\n"%df_test.shape[0])
print("Omni NLP API\n") 
print("Total of right answers: %d"%omni_right_answer)
omni_acc = round(omni_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %.2f\n"%omni_acc)

print("Watson Assistent\n") 
print("Total of right answers: %d"%watson_right_answer)
watson_acc = round(watson_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %.2f"%watson_acc)

-------------------------------------------
sentence: quero ir numa agência da CEDAE


Omni NLP API

predict_intent: Realizar_Denuncia
score: 0.69
right_intent: Localizar_AgenciasCEDAE
is it right?: no
quantity of right answers: 0


Watson Assistent

predict_intent: Localizar_AgenciasCEDAE
score: 0.91
right_intent: Localizar_AgenciasCEDAE
is it right?: yes
quantity of right answers: 1


-------------------------------------------
sentence: Boa Noite


Omni NLP API

predict_intent: anything_else
score: 0.10
right_intent: General_Greetings
is it right?: no
quantity of right answers: 0


Watson Assistent

predict_intent: failed to identify
score: -1.00
right_intent: General_Greetings
is it right?: no
quantity of right answers: 1


-------------------------------------------
sentence: quero um relatório dos meus pagamentos


Omni NLP API

predict_intent: Realizar_Denuncia
score: 0.46
right_intent: Consultar_ContasPagas
is it right?: no
quantity of right answers: 0


Watson Assistent

predi

-------------------------------------------
sentence: Desejo denunciar uma ligação irregular de água


Omni NLP API

predict_intent: Reportar_FaltaDeAgua
score: 0.73
right_intent: Realizar_Denuncia
is it right?: no
quantity of right answers: 6


Watson Assistent

predict_intent: Realizar_Denuncia
score: 0.99
right_intent: Realizar_Denuncia
is it right?: yes
quantity of right answers: 16


-------------------------------------------
sentence: Quero saber se tenho dívidas com a CEDAE


Omni NLP API

predict_intent: Solicitar_Contato
score: 0.74
right_intent: Emitir_NadaConsta
is it right?: no
quantity of right answers: 6


Watson Assistent

predict_intent: Emitir_NadaConsta
score: 0.59
right_intent: Emitir_NadaConsta
is it right?: yes
quantity of right answers: 17


-------------------------------------------
sentence: como faço para encontrar vocês?


Omni NLP API

predict_intent: Solicitar_Contato
score: 0.46
right_intent: Localizar_AgenciasCEDAE
is it right?: no
quantity of right answ

-------------------------------------------
sentence: Debito em meu nome


Omni NLP API

predict_intent: Parcelar_Conta
score: 0.66
right_intent: Emitir_NadaConsta
is it right?: no
quantity of right answers: 11


Watson Assistent

predict_intent: Parcelar_Conta
score: 0.53
right_intent: Emitir_NadaConsta
is it right?: no
quantity of right answers: 32


-------------------------------------------
sentence: obrigada


Omni NLP API

predict_intent: anything_else
score: 0.10
right_intent: General_Ending
is it right?: no
quantity of right answers: 11


Watson Assistent

predict_intent: failed to identify
score: -1.00
right_intent: General_Ending
is it right?: no
quantity of right answers: 32


-------------------------------------------
sentence: obrigado


Omni NLP API

predict_intent: anything_else
score: 0.10
right_intent: General_Ending
is it right?: no
quantity of right answers: 11


Watson Assistent

predict_intent: General_Ending
score: 0.67
right_intent: General_Ending
is it right?:

-------------------------------------------
sentence: Estou com uma conta atrasada como faço para negociar


Omni NLP API

predict_intent: General_Negative_Feedback
score: 0.71
right_intent: Parcelar_Conta
is it right?: no
quantity of right answers: 18


Watson Assistent

predict_intent: Emitir_SegundaViaConta
score: 0.61
right_intent: Parcelar_Conta
is it right?: no
quantity of right answers: 47


-------------------------------------------
sentence: tenho um vazamento na conexão do meu relógio


Omni NLP API

predict_intent: Consultar_MatriculaCPF
score: 0.70
right_intent: Reportar_Vazamento
is it right?: no
quantity of right answers: 18


Watson Assistent

predict_intent: Reportar_Vazamento
score: 0.95
right_intent: Reportar_Vazamento
is it right?: yes
quantity of right answers: 48


-------------------------------------------
sentence: Esqueci o meu CPF


Omni NLP API

predict_intent: Parcelar_Conta
score: 0.60
right_intent: Consultar_MatriculaCPF
is it right?: no
quantity of right

-------------------------------------------
sentence: Como faço para falar com a Cedae?


Omni NLP API

predict_intent: Solicitar_Contato
score: 0.80
right_intent: Solicitar_Contato
is it right?: yes
quantity of right answers: 24


Watson Assistent

predict_intent: Solicitar_Contato
score: 0.68
right_intent: Solicitar_Contato
is it right?: yes
quantity of right answers: 64


-------------------------------------------
sentence: Você tem um nome?


Omni NLP API

predict_intent: Bot_About
score: 0.67
right_intent: Bot_About
is it right?: yes
quantity of right answers: 25


Watson Assistent

predict_intent: Bot_About
score: 0.76
right_intent: Bot_About
is it right?: yes
quantity of right answers: 65


-------------------------------------------
sentence: Eu que agradeço


Omni NLP API

predict_intent: Bot_Capabilities
score: 0.00
right_intent: General_Positive_Feedback
is it right?: no
quantity of right answers: 25


Watson Assistent

predict_intent: General_Ending
score: 0.50
right_inten

In [9]:
row.example

'quero ir numa agência da CEDAE'